In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

C:\Users\slyx183\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
##########start of parser###############

In [3]:
df_raw = pd.read_excel('VGCPastes Repository.xlsx', header = 2)

In [4]:
df_raw = df_raw[df_raw['Original Paste'] != '-']

In [5]:
def remove_nickname(string):
    if '(' in string and ')' in string:
        new_name = string[string.index('(')+1:string.index(')')]
        if len(new_name) > 1:
            return new_name
        else:
            return string
    else:
        return string

In [6]:
urls = df_raw['Original Paste']

In [7]:
df_compile = pd.DataFrame(columns=[0,1,2,3,4,5,6])
for URL in urls: #URL = "https://pokepast.es/7439f2100cd4871a" # "https://realpython.github.io/fake-jobs/"
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find_all('article')

    df = pd.DataFrame()

    team_set = []
    title = soup.find('aside').text.strip()
    split_title = title.split("Columns", 1)
    team_set.append(URL)
    team_set.append(split_title[0])
    pkmn_set = []
    for result in results:
        try:
            pkmn_text = result.text.strip()
            split_pkmn = pkmn_text.split("- ")
            try:
                moves = split_pkmn[1].strip(),split_pkmn[2].strip(),split_pkmn[3].strip(),split_pkmn[4].strip()
            except:
                try:
                    moves = split_pkmn[1].strip()
                except:
                    try:
                        moves = split_pkmn[1].strip(),split_pkmn[2].strip(),split_pkmn[3]
                    except:
                        moves = split_pkmn[1].strip(),split_pkmn[2].strip()
            name_item_misc = split_pkmn[0].split(" @ ")
            name_item_misc
            name = name_item_misc[0].strip()
            name = name.replace(" (M)",'')
            name = name.replace(" (F)",'')
            name = name.replace("-Gmax",'')
            name = remove_nickname(name)
            name = name.strip()
            item_misc = name_item_misc[1].split(" \n",1)
            item = item_misc[0].strip()
            misc = item_misc[1].strip()
            team_set.append(name)
            team_set.append(item)
            team_set.append(moves)
            team_set.append(misc)
        except:
            pass
    df['0'] = team_set
    df_t = df.transpose()
    df_compile = df_compile.append(df_t, ignore_index = True)


In [8]:
df_compile.columns = ['url','Title', 'Pokemon 1','Item 1','Moves 1','Misc 1',
                'Pokemon 2','Item 2','Moves 2','Misc 2', 
                'Pokemon 3','Item 3','Moves 3','Misc 3', 
                'Pokemon 4','Item 4','Moves 4','Misc 4', 
                'Pokemon 5','Item 5','Moves 5','Misc 5', 
                'Pokemon 6','Item 6','Moves 6','Misc 6']

In [9]:
df_compile['Full_Team'] = df_compile['Pokemon 1']+df_compile['Pokemon 2']+df_compile['Pokemon 3']+df_compile['Pokemon 4']+df_compile['Pokemon 5']+df_compile['Pokemon 6']
df_compile.to_excel('vgc_pastes_dump.xlsx',index =False)

In [ ]:
##########end of parser, start of analytics############

In [10]:
##########end of parser, start of analytics############
df_compile = pd.read_excel('vgc_pastes_dump.xlsx')

In [11]:
df_pkmn_item_moves_compile = pd.DataFrame(columns = ['Pokemon','Item','Moves'])
for i in range(1,7):
    df_compile['Pokemon'] = df_compile['Pokemon '+str(i)]
    df_compile['Item'] = df_compile['Item '+str(i)]
    df_compile['Moves'] = df_compile['Moves '+str(i)].str.strip('()')
    df_pkmn_item_moves_compile = df_pkmn_item_moves_compile[['Pokemon','Item','Moves']].append(df_compile[['Pokemon','Item','Moves']], ignore_index = True)
    df_pkmn_item_moves_compile[['Move 1','Move 2','Move 3','Move 4']] = df_pkmn_item_moves_compile['Moves'].str.split(',', expand=True)
    for j in range(1,5):
        df_pkmn_item_moves_compile['Move '+str(j)] = df_pkmn_item_moves_compile['Move '+str(j)].str.replace('"','')
        df_pkmn_item_moves_compile['Move '+str(j)] = df_pkmn_item_moves_compile['Move '+str(j)].str.replace("'",'')

In [12]:
df_pkmn_moves_compile = pd.DataFrame(columns = ['Pokemon','Move'])
for i in range(1,5):
    df_pkmn_item_moves_compile['Pokemon'] = df_pkmn_item_moves_compile['Pokemon']
    df_pkmn_item_moves_compile['Move'] = df_pkmn_item_moves_compile['Move '+str(i)]
    df_pkmn_item_moves_compile['Move'] = df_pkmn_item_moves_compile['Move'].str.strip()
    df_pkmn_moves_compile = df_pkmn_moves_compile[['Pokemon','Move']].append(df_pkmn_item_moves_compile[['Pokemon','Move']], ignore_index = True)

In [13]:
target = 'Grimmsnarl'
df_target_pkmn = df_pkmn_item_moves_compile[df_pkmn_item_moves_compile['Pokemon'].str.contains(target) == True]
df_target_pkmn_item = df_target_pkmn[['Pokemon','Item']]
df_target_pkmn_item['Count'] = 1
item_breakdown = df_target_pkmn_item.groupby(['Pokemon','Item']).count().reset_index()

<ipython-input-13-038968047b1b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_pkmn_item['Count'] = 1


In [14]:
item_breakdown.sort_values(by='Count', ascending=False)

,Pokemon,Item,Count
10,Grimmsnarl,Light Clay,172
3,Grimmsnarl,Focus Sash,18
5,Grimmsnarl,Iron Ball,16
0,Grimmsnarl,Babiri Berry,13
6,Grimmsnarl,Lagging Tail,10
1,Grimmsnarl,Black Glasses,3
2,Grimmsnarl,Eject Button,2
4,Grimmsnarl,Full Incense,2
7,Grimmsnarl,Lax Incense,1
8,Grimmsnarl,Life Orb,1


In [17]:
target = 'Fake Out'
df_target_pkmn = df_pkmn_moves_compile[df_pkmn_moves_compile['Move'].str.contains(target) == True]
df_target_pkmn_move = df_target_pkmn[['Pokemon','Move']]
df_target_pkmn_move['Count'] = 1
move_breakdown = df_target_pkmn_move.groupby(['Pokemon','Move']).count().reset_index()

In [18]:
move_breakdown.sort_values(by='Count', ascending=False)

,Pokemon,Move,Count
5,Incineroar,Fake Out,531
20,Rillaboom,Fake Out,195
0,Blastoise,Fake Out,36
3,Grimmsnarl,Fake Out,32
19,Raichu,Fake Out,27
2,Gothitelle,Fake Out,10
21,Sableye,Fake Out,7
18,Pikachu,Fake Out,7
17,Persian-Alola,Fake Out,5
13,Mienshao,Fake Out,5


In [29]:
##########end of parser, start of searcher############
df_compile = pd.read_excel('vgc_pastes_dump.xlsx')

In [30]:
def search_fn(search_type,search_param_1,search_param_2,search_param_3,set_param_1,set_param_2,set_param_3):
    if search_type == 'pkmn':
        print('Searching pokemon with condition(s) ' + search_param_1 + ',' + search_param_2+ ',' + search_param_3  + ' out of pokemon with condition(s) '+ set_param_1 + ',' + set_param_2+ ',' + set_param_3 + ': '+ 
            str(len(df_compile[((df_compile['Pokemon 1'].str.contains(search_param_1) == True)
                   &(df_compile['Item 1'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 1'].str.contains(search_param_3) == True))
                  |((df_compile['Pokemon 2'].str.contains(search_param_1) == True)
                   &(df_compile['Item 2'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 2'].str.contains(search_param_3) == True))
                  |((df_compile['Pokemon 3'].str.contains(search_param_1) == True)
                   &(df_compile['Item 3'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 3'].str.contains(search_param_3) == True))
                  |((df_compile['Pokemon 4'].str.contains(search_param_1) == True)
                   &(df_compile['Item 4'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 4'].str.contains(search_param_3) == True))
                  |((df_compile['Pokemon 5'].str.contains(search_param_1) == True)
                   &(df_compile['Item 5'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 5'].str.contains(search_param_3) == True))
                  |((df_compile['Pokemon 6'].str.contains(search_param_1) == True)
                   &(df_compile['Item 6'].str.contains(search_param_2) == True)
                   &(df_compile['Moves 6'].str.contains(search_param_3) == True))])/
                len(df_compile[((df_compile['Pokemon 1'].str.contains(set_param_1) == True)
                   &(df_compile['Item 1'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 1'].str.contains(set_param_3) == True))
                  |((df_compile['Pokemon 2'].str.contains(set_param_1) == True)
                   &(df_compile['Item 2'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 2'].str.contains(set_param_3) == True))
                  |((df_compile['Pokemon 3'].str.contains(set_param_1) == True)
                   &(df_compile['Item 3'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 3'].str.contains(set_param_3) == True))
                  |((df_compile['Pokemon 4'].str.contains(set_param_1) == True)
                   &(df_compile['Item 4'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 4'].str.contains(set_param_3) == True))
                  |((df_compile['Pokemon 5'].str.contains(set_param_1) == True)
                   &(df_compile['Item 5'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 5'].str.contains(set_param_3) == True))
                  |((df_compile['Pokemon 6'].str.contains(set_param_1) == True)
                   &(df_compile['Item 6'].str.contains(set_param_2) == True)
                   &(df_compile['Moves 6'].str.contains(set_param_3) == True))])*100)+'%')
    elif search_type == 'team':
        print('Searching teams with condition(s) ' + search_param_1 + ',' + search_param_2+ ',' + search_param_3 
              + ' out of teams with condition(s) '+ set_param_1 + ',' + set_param_2+ ',' + set_param_3 + ': '+ 
              str(len(df_compile[((df_compile['Full_Team'].str.contains(search_param_1) == True)
                   &(df_compile['Full_Team'].str.contains(search_param_2) == True)
                   &(df_compile['Full_Team'].str.contains(search_param_3) == True))
                  ])/
                len(df_compile[((df_compile['Full_Team'].str.contains(set_param_1) == True)
                   &(df_compile['Full_Team'].str.contains(set_param_2) == True)
                   &(df_compile['Full_Team'].str.contains(set_param_3) == True))
                  ])*100)+'%')
    else:
        print('Error - Please enter valid search type (pkmn or team)')

In [115]:
###toggle between pkmn (search pokemon with X attributes) or team (search team with X and Y pkmn)###
search_type = 'team' 
###toggle between pkmn (search pokemon with X attributes) or team (search team with X and Y pkmn)###

###leave '' in params if not needed###
search_param_1 = 'Calyrex-Ice'
search_param_2 = 'Incineroar'
search_param_3 = '' 

set_param_1 = 'Calyrex-Ice'
set_param_2 = ''
set_param_3 = ''
###leave spaces in params if not needed###
search_fn(search_type,search_param_1,search_param_2,search_param_3,set_param_1,set_param_2,set_param_3)

Searching teams with condition(s) Calyrex-Ice,Incineroar, out of teams with condition(s) Calyrex-Ice,,: 73.95833333333334%
